 # Capstone - Battle of Neighborhoods Part2

## Importing various Python Libraries

In this section we import the libraries that will be required to process the data.

Import the libraries to set up and analyse data for the Python programming language.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Python Libraries import successful')

Solving environment: | 

## Download and Explore Dataset


Download and Explore Dataset
location has a total of 5 boroughs and 306 cluster_areas. In order to segement the cluster_areas and explore them, we will essentially need a dataset that contains the 5 boroughs and the cluster_areas that exist in each borough as well as the the latitude and logitude coordinates of each location.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

Downloaded the files and place it on the server, so run a wget command and access the data.

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


#### Tranform the data into a *pandas* dataframe

In [6]:
cluster_areas_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'location', 'Latitude', 'Longitude'] 

# instantiate the dataframe
cluster_areas = pd.DataFrame(columns=column_names)

for data in cluster_areas_data:
    borough = location_name = data['properties']['borough'] 
    location_name = data['properties']['name']
        
    location_latlon = data['geometry']['coordinates']
    location_lat = location_latlon[1]
    location_lon = location_latlon[0]
    
    cluster_areas = cluster_areas.append({'Borough': borough,
                                          'location': location_name,
                                          'Latitude': location_lat,
                                          'Longitude': location_lon}, ignore_index=True)

In [7]:
cluster_areas.head()

,Borough,location,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with cluster_areas superimposed on top.

In [9]:
manhattan_data = cluster_areas[cluster_areas['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,location,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [11]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, location in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['location']):
    label = '{}, {}'.format(location, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues


In [29]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['location', 
                  'location Latitude', 
                  'location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [30]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'K3GUGBP3YLE35FIGZVRRHFAFRKQ5RITNWDHORVZ1AQBM2QG0'
CLIENT_SECRET = 'HKXAT34M2PHDPNFU4PONKBL5TUNA0BEF0L5KFJHSO1IRWCPL'
VERSION = '20181020'

In [31]:
#https://developer.foursquare.com/docs/resources/categories
#Food_truck = 4bf58dd8d48988d1cb941735
cluster_areas = cluster_areas[cluster_areas['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_Food_truck = getNearbyVenues(names=cluster_areas['location'], latitudes=cluster_areas['Latitude'], longitudes=cluster_areas['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1cb941735')
newyork_venues_Food_truck.head()

,location,location Latitude,location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Parada Tropical,40.872606,-73.905942,Food Truck
1,Marble Hill,40.876551,-73.91066,Tony's Halal,40.874225,-73.894499,Food Truck
2,Marble Hill,40.876551,-73.91066,Ice Cream Truck,40.872010,-73.918699,Food Truck
3,Marble Hill,40.876551,-73.91066,Monkey Brothers Lab,40.867667,-73.917075,Food Truck
4,Marble Hill,40.876551,-73.91066,Mexican Food Truck,40.871658,-73.897858,Food Truck


In [32]:
newyork_venues_Food_truck.shape

(1830, 7)

In [33]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['location'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [34]:
map_newyork_Food_truck = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_Food_truck, 'purple', map_newyork_Food_truck)

map_newyork_Food_truck

In [35]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('location').count()
    
    for n in startDf['location']:
        try:
            startDf.loc[startDf['location'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['location'] == n,columnTitle] = 0

In [36]:
manhattan_grouped = newyork_venues_Food_truck.groupby('location').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_Food_truck['Venue Category'].unique())))

,location Latitude,location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
location,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,50,50,50,50,50,50
Central Harlem,32,32,32,32,32,32
Chelsea,50,50,50,50,50,50
Chinatown,49,49,49,49,49,49
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,42,42,42,42,42,42
East Village,49,49,49,49,49,49


## 3. Analyze Each location

In [37]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_Food_truck[['Venue Category']], prefix="", prefix_sep="")

# add location column back to dataframe
manhattan_onehot['location'] = newyork_venues_Food_truck['location'] 

# move location column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,location,Afghan Restaurant,American Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Caribbean Restaurant,Coffee Shop,Cupcake Shop,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Food,Food Court,Food Stand,Food Truck,Greek Restaurant,Halal Restaurant,Ice Cream Shop,Indian Restaurant,Latin American Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Restaurant,Taco Place
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [38]:
manhattan_grouped = manhattan_onehot.groupby('location').mean().reset_index()
manhattan_grouped

,location,Afghan Restaurant,American Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Caribbean Restaurant,Coffee Shop,Cupcake Shop,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Food,Food Court,Food Stand,Food Truck,Greek Restaurant,Halal Restaurant,Ice Cream Shop,Indian Restaurant,Latin American Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Restaurant,Taco Place
0,Battery Park City,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.980000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,Carnegie Hill,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.020000,0.00000,0.000000,0.000000,0.900000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.02000
2,Central Harlem,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.03125,0.000000,0.000000,0.968750,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,Chelsea,0.02,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.940000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.00000
4,Chinatown,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.020408,0.020408,0.00,0.000000,0.000000,0.00000,0.020408,0.020408,0.897959,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.00000
5,Civic Center,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.00000,0.020000,0.020000,0.940000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,Clinton,0.00,0.02,0.00,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.00000,0.000000,0.000000,0.860000,0.020000,0.00,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.020000,0.000000,0.00000
7,East Harlem,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.023810,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.880952,0.000000,0.00,0.047619,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.02381
8,East Village,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
9,Financial District,0.00,0.00,0.00,0.000000,0.000000,0.02,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.960000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Best neighbourhood'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Best neighbourhood'.format(ind+1))

# create a new dataframe
cluster_areas_venues_sorted = pd.DataFrame(columns=columns)
cluster_areas_venues_sorted['location'] = manhattan_grouped['location']

for ind in np.arange(manhattan_grouped.shape[0]):
    cluster_areas_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

cluster_areas_venues_sorted.head()

,location,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
0,Battery Park City,Food Truck,Coffee Shop,Taco Place,Food,American Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Caribbean Restaurant
1,Carnegie Hill,Food Truck,Taco Place,Farmers Market,Ice Cream Shop,Coffee Shop,Restaurant,American Restaurant,BBQ Joint,Bakery,Breakfast Spot
2,Central Harlem,Food Truck,Food,Taco Place,American Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Caribbean Restaurant,Coffee Shop
3,Chelsea,Food Truck,Afghan Restaurant,Mexican Restaurant,Lebanese Restaurant,Farmers Market,American Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint
4,Chinatown,Food Truck,Food Court,Middle Eastern Restaurant,Coffee Shop,Cupcake Shop,Food Stand,Farmers Market,American Restaurant,BBQ Joint,Bakery


Cluster cluster_areas


In [39]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 2, 0, 2, 2, 1, 4, 2], dtype=int32)

In [40]:
# add clustering labels
cluster_areas_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(cluster_areas_venues_sorted.set_index('location'), on='location')

manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
0,Manhattan,Marble Hill,40.876551,-73.910660,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
1,Manhattan,Chinatown,40.715618,-73.994279,0.0,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck


In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['location'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

In [42]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
1,Chinatown,Indian Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
8,Upper East Side,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
9,Yorkville,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
10,Lenox Hill,Indian Restaurant,North Indian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant
11,Roosevelt Island,Indian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant
12,Upper West Side,Indian Restaurant,South Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
13,Lincoln Square,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant
18,Greenwich Village,Indian Restaurant,North Indian Restaurant,Food Truck,Dosa Place,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
20,Lower East Side,Indian Restaurant,Vegetarian / Vegan Restaurant,North Indian Restaurant,Deli / Bodega,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
23,Soho,Indian Restaurant,Dosa Place,North Indian Restaurant,Food Truck,Deli / Bodega,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant


In [43]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
0,Marble Hill,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
2,Washington Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
4,Hamilton Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
5,Manhattanville,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
6,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
7,East Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Himalayan Restaurant,Food Truck
25,Manhattan Valley,Indian Restaurant,Himalayan Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Food Truck
26,Morningside Heights,Indian Restaurant,Himalayan Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar,Food Truck


In [44]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
14,Clinton,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
17,Chelsea,Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Hookah Bar,Himalayan Restaurant
28,Battery Park City,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
29,Financial District,Indian Restaurant,Food Truck,Tapas Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar
32,Civic Center,Indian Restaurant,Food Truck,Tapas Restaurant,Dosa Place,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant
39,Hudson Yards,Indian Restaurant,Food Truck,South Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant,Hookah Bar


In [45]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
37,Stuyvesant Town,Indian Restaurant,Vegetarian / Vegan Restaurant,North Indian Restaurant,Dosa Place,Deli / Bodega,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant,Indian Chinese Restaurant


In [46]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Best neighbourhood,2nd Best neighbourhood,3rd Best neighbourhood,4th Best neighbourhood,5th Best neighbourhood,6th Best neighbourhood,7th Best neighbourhood,8th Best neighbourhood,9th Best neighbourhood,10th Best neighbourhood
15,Midtown,Indian Restaurant,Food Truck,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Diner,Vegetarian / Vegan Restaurant,Tapas Restaurant,Indian Chinese Restaurant,Hookah Bar
16,Murray Hill,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,North Indian Restaurant,Diner,Tapas Restaurant,Pakistani Restaurant,Indian Chinese Restaurant,Hookah Bar
19,East Village,Indian Restaurant,Dosa Place,Deli / Bodega,Vegetarian / Vegan Restaurant,North Indian Restaurant,Food Truck,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
21,Tribeca,Indian Restaurant,Food Truck,Tapas Restaurant,Dosa Place,Asian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant
22,Little Italy,Indian Restaurant,Dosa Place,North Indian Restaurant,Food Truck,Deli / Bodega,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
27,Gramercy,Indian Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,North Indian Restaurant,Hookah Bar,Food Truck,Deli / Bodega,Chaat Place,Burrito Place,Tapas Restaurant
31,Noho,Indian Restaurant,Dosa Place,Vegetarian / Vegan Restaurant,North Indian Restaurant,Food Truck,Deli / Bodega,Chaat Place,Tapas Restaurant,South Indian Restaurant,Pakistani Restaurant
33,Midtown South,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,Diner,Burrito Place,Tapas Restaurant,Pakistani Restaurant,North Indian Restaurant,Indian Chinese Restaurant
38,Flatiron,Indian Restaurant,South Indian Restaurant,Food Truck,Vegetarian / Vegan Restaurant,North Indian Restaurant,Hookah Bar,Diner,Deli / Bodega,Burrito Place,Tapas Restaurant
